<a href="https://colab.research.google.com/github/stevensiadaty/musicformer/blob/main/ColabVerGenerateMusicWithAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate music, given neural net
## Piano, single track
### Based on Google 2018 Music Transformer NN

Codes recycled from:

1) Alex https://github.com/asigalov61/SuperPiano/blob/master/Super_Piano_3.ipynb

2) Damon https://github.com/gwinndr/MusicTransformer-Pytorch

3) Jason https://github.com/jason9693/midi-neural-processor

4) Mir https://github.com/mirsiadaty

Thank you :)


## Import Time

from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
import time
!pip install pretty_midi
from google.colab import drive
drive.mount('/content/gdrive')

print(time.asctime( time.localtime( time.time() ) ))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 17.2 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=2e5c985bdf5b830495ef2374a4e61cce65330368d3b8ee33604e3e1b1bfcc966
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
Mounted at /content/gdrive
Sun Sep  4 23:11:56 2022


# Create dir for this project

In [4]:
# colab
YourHomeDir = '/content/'
YourProjectSubDir = 'gen1_1'


!mkdir $YourHomeDir/$YourProjectSubDir
%cd $YourHomeDir/$YourProjectSubDir
!ls -ltA 

#======================================

#!git clone https://github.com/asigalov61/midi-neural-processor
!git clone https://github.com/jason9693/midi-neural-processor

#!git clone https://github.com/asigalov61/MusicTransformer-Pytorch
!git clone https://github.com/gwinndr/MusicTransformer-Pytorch

#======================================
    
%cd $YourHomeDir/$YourProjectSubDir

!mkdir MusicTransformer-Pytorch/rpr
!mkdir MusicTransformer-Pytorch/rpr/results


#!wget 'https://raw.githubusercontent.com/stevensiadaty/musicformer/main/SP3_20220830_0807.ipynb'
#select_model = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle" 
#github says: Yowza, that’s a big file. Try again with a file smaller than 25MB. 

%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/rpr/results/

!wget 'https://raw.githubusercontent.com/stevensiadaty/musicformer/main/best_loss_weights.pickle'    

#======================================

%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/dataset/

!wget 'https://raw.githubusercontent.com/stevensiadaty/musicformer/main/e_piano.zip'\

!unzip e_piano.zip


#%cd /content/
%cd $YourHomeDir/$YourProjectSubDir

# this is a renaming!
!mv midi-neural-processor midi_processor

# do for code preprocess_midi.py : import third_party.midi_processor.processor as midi_processor
!cp -r midi_processor/* $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/third_party/midi_processor/


#%cd /content/MusicTransformer-Pytorch/
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/

#======================================

#%cd /content/
%cd $YourHomeDir/$YourProjectSubDir

# this is a renaming!
!mv midi-neural-processor midi_processor

# do for code preprocess_midi.py : import third_party.midi_processor.processor as midi_processor
!cp -r midi_processor/* $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/third_party/midi_processor/


#%cd /content/MusicTransformer-Pytorch/
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/

#======================================

print(time.asctime( time.localtime( time.time() ) ))

/content/gen1_1
total 0
Cloning into 'midi-neural-processor'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (27/27), done.
Cloning into 'MusicTransformer-Pytorch'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 346 (delta 5), reused 13 (delta 5), pack-reused 332
Receiving objects: 100% (346/346), 109.42 KiB | 15.63 MiB/s, done.
Resolving deltas: 100% (190/190), done.
/content/gen1_1
/content/gen1_1/MusicTransformer-Pytorch/rpr/results
--2022-09-04 23:13:03--  https://raw.githubusercontent.com/stevensiadaty/musicformer/main/best_loss_weights.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

## 71: Assign values for parameters of music generation

In [5]:
#@title Generate, Plot, Graph, Save, Download, and Render the resulting output
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/

number_of_tokens_to_generate = 2048 #@param {type:"slider", min:1, max:2048, step:1}
priming_sequence_length = 256 #@param {type:"slider", min:1, max:2048, step:8}
maximum_possible_output_sequence = 2048 #@param {type:"slider", min:0, max:2048, step:8}

#@param ["/content/MusicTransformer-Pytorch/rpr/results/best_acc_weights.pickle", "/content/MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle"]
select_model = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle" 

custom_MIDI = "" #@param {type:"string"}


# get to the right dir
%cd $YourHomeDir/$YourProjectSubDir/midi_processor
#
from google.colab import files


import processor
from processor import encode_midi, decode_midi

print(time.asctime( time.localtime( time.time() ) ))

/content/gen1_1/MusicTransformer-Pytorch
/content/gen1_1/midi_processor
Sun Sep  4 23:13:27 2022


## 72: Execute the generation code: make music! (repeat as many as you like)

In [6]:
# auto-time-stamped subdir to save the music
DirToSaveGeneratedMusic = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/generated/" + str(int(time.time()))


# get to the right dir
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch

# time keeping
startt = time.asctime( time.localtime( time.time() ) )

!python3 generate.py -output_dir $DirToSaveGeneratedMusic -model_weights=$select_model --rpr \
 -target_seq_length=$number_of_tokens_to_generate -num_prime=$priming_sequence_length \
 -max_sequence=$maximum_possible_output_sequence $custom_MIDI #

print('Successfully exported the output to output folder. To primer.mid and rand.mid')

print(startt)
print(time.asctime( time.localtime( time.time() ) ))

/content/gen1_1/MusicTransformer-Pytorch
midi_root: ./dataset/e_piano/
output_dir: /content//gen1_1/MusicTransformer-Pytorch/generated/1662333212
primer_file: None
force_cpu: False

target_seq_length: 2048
num_prime: 256
model_weights: /content//gen1_1/MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle
beam: 0

rpr: True
max_sequence: 2048
n_layers: 6
num_heads: 8
d_model: 512

dim_feedforward: 1024

Using primer index: 159 ( ./dataset/e_piano/test/MIDI-Unprocessed_24_R1_2006_01-05_ORIG_MID--AUDIO_24_R1_2006_03_Track03_wav.midi.pickle )
RAND DIST
Generating sequence of max length: 2048
300 / 2048
350 / 2048
400 / 2048
450 / 2048
500 / 2048
550 / 2048
600 / 2048
650 / 2048
700 / 2048
750 / 2048
800 / 2048
850 / 2048
900 / 2048
950 / 2048
1000 / 2048
1050 / 2048
1100 / 2048
1150 / 2048
1200 / 2048
1250 / 2048
1300 / 2048
1350 / 2048
1400 / 2048
1450 / 2048
1500 / 2048
1550 / 2048
1600 / 2048
1650 / 2048
1700 / 2048
1750 / 2048
1800 / 2048
1850 / 2048
1900 / 2048
1950 / 2048
20

## Play Audio (WIP)

In [ ]:
!pip install Ipython
!pip install pretty_midi
!pip install music
!pip install pyfluidsynth

print(time.asctime( time.localtime( time.time() ) ))

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3 -m pip install --upgrade pip
Sat Sep  3 17:30:21 2022


In [ ]:
from IPython.display import Audio
from pretty_midi import PrettyMIDI

sf2_path = '/home/stsi28/Downloads/piano.sf2'  # path to sound font file
midi_file = '/home/stsi28/gen1_4/MusicTransformer-Pytorch/generated/1662237664/rand.mid'

music = PrettyMIDI(midi_file=midi_file)
waveform = music.fluidsynth(sf2_path=sf2_path)
#Audio(waveform, rate=44100)

print(time.asctime( time.localtime( time.time() ) ))

ImportError: fluidsynth() was called but pyfluidsynth is not installed.

## 73: List the music files generated

In [ ]:
!ls -ltAR $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/generated/

print(time.asctime( time.localtime( time.time() ) ))

/home/stsi28//gen1_4/MusicTransformer-Pytorch/generated/:
total 4
drwxrwxr-x 2 stsi28 stsi28 4096 Sep  3 16:42 1662237664

/home/stsi28//gen1_4/MusicTransformer-Pytorch/generated/1662237664:
total 8
-rw-rw-r-- 1 stsi28 stsi28 3053 Sep  3 16:42 rand.mid
-rw-rw-r-- 1 stsi28 stsi28  429 Sep  3 16:41 primer.mid
Sat Sep  3 16:44:31 2022
